In [1]:
import json
from tqdm import tqdm
import torch
from sentence_transformers import util

In [2]:
big = "memoryalpha"
small = "stexpanded"

mapping_file_big = "./data/triples_v2/" + big + "_mapping.json"
mapping_file_small = "./data/triples_v2/" + small + "_mapping.json"

communities_big_file = "./results/communities_leiden/" + big + "/final.txt"
communitiy_embeddings_big_file = "./results/community_embeddings/" + big + ".json"

node_embeddings_small_file = "./results/embeddings/" + small + "_lab_altlab_type_abs_comment_BAAI_bge-large-en-v1.5.json"
url_embeddings_small_file = "./results/url_embeddings_short/" + small + "_url_BAAI_bge-large-en-v1.5.json"

exact_match_file = "./data/exact_match/" + big + "-" + small + "-formatted.json"
gold_pairs_file = "./data/gold_pairs/" + big + "-" + small + "_formatted.txt"

top10pairs_file = "./results/top10pairs/" + small + "-" + big + "_top10pairs.json"

In [3]:
big_communities = []
gold_pairs = []

with open(communities_big_file) as cbf, open(gold_pairs_file) as gpf:

    for line in cbf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        big_communities.append(numbers_set)

    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

with open(exact_match_file) as file:
    exact_match = json.load(file)

gold_not_exact = list()
for p in gold_pairs:
    if [p[0], p[1]] not in exact_match:
        gold_not_exact.append([p[0], p[1]])

with open(communitiy_embeddings_big_file) as cebf:
    community_embeddings_big = json.load(cebf)
    community_embeddings_big = {k: list(v.values()) for k, v in community_embeddings_big.items()}

with open(top10pairs_file) as file:
    top10pairs = json.load(file)

with open(node_embeddings_small_file) as nesf:
    node_embeddings_small = json.load(nesf)

with open(url_embeddings_small_file) as uesf:
    url_embeddings_small = json.load(uesf)

merged_node_embeddings_small = {key: node_embeddings_small[key] if key in node_embeddings_small else url_embeddings_small[key] for key in set(node_embeddings_small) | set(url_embeddings_small)}
merged_node_embeddings_small = {str(k): merged_node_embeddings_small[str(k)] for k in sorted(map(int, merged_node_embeddings_small.keys()))}

In [4]:
top1dict = dict()

for k, v in top10pairs.items():
    top1dict[int(k)] = int(v[0][0])

In [5]:
found = 0
all_pairs = len(gold_pairs)
# all_pairs = len(gold_not_exact)

for gold_pair in tqdm(gold_pairs):
# for gold_pair in tqdm(gold_not_exact):
    if top1dict.get(gold_pair[1]) == gold_pair[0]:
        found += 1
    # else:
    #     print(gold_pair)

print("Gold pairs in top 1: " + str(found / all_pairs * 100) + "%")

100%|██████████| 1779/1779 [00:00<00:00, 1135372.31it/s]

Gold pairs in top 1: 78.13378302417088%


In [6]:
node_to_community_embeddings = dict()
index = 0
for community in big_communities:
    for node in community:
        node_to_community_embeddings[node] = community_embeddings_big[str(index)]
    index += 1

In [ ]:
#TODO

In [32]:
node_embeds = torch.Tensor(merged_node_embeddings_small['2440'])
compare_list = [node_to_community_embeddings[int(item[0])] for item in top10pairs['0']]
big_torch_embeds = torch.Tensor(compare_list)

community_order = util.semantic_search(node_embeds, big_torch_embeds)

In [33]:
top10pairs['2440']

[['73628', 0.7748620510101318],
 ['34463', 0.7659156918525696],
 ['139583', 0.710580050945282],
 ['92236', 0.7008298635482788],
 ['22527', 0.6984464526176453],
 ['93865', 0.6762269735336304],
 ['15660', 0.6759642362594604],
 ['54670', 0.6698982119560242],
 ['9430', 0.6697186827659607],
 ['67271', 0.6663190722465515]]

In [34]:
community_order

[[{'corpus_id': 8, 'score': 0.6746936440467834},
  {'corpus_id': 1, 'score': 0.6725383996963501},
  {'corpus_id': 9, 'score': 0.6617224216461182},
  {'corpus_id': 5, 'score': 0.6589083671569824},
  {'corpus_id': 4, 'score': 0.652053713798523},
  {'corpus_id': 2, 'score': 0.6348870992660522},
  {'corpus_id': 7, 'score': 0.6225403547286987},
  {'corpus_id': 0, 'score': 0.6225403547286987},
  {'corpus_id': 6, 'score': 0.6205182075500488},
  {'corpus_id': 3, 'score': 0.6166427135467529}]]

In [38]:
top1dict_reordered = dict()
best_score = 0

for item in community_order[0]:
    score = top10pairs['2440'][item['corpus_id']][1] + 0.5 * item['score']
    id_node = top10pairs['2440'][item['corpus_id']][0]
    print(id_node, score)
    if score > best_score:
        best_score = score
        top1dict_reordered[int('2440')] = int(id_node)

9430 1.0070655047893524
34463 1.1021848917007446
67271 0.9971802830696106
93865 1.0056811571121216
22527 1.0244733095169067
139583 1.028023600578308
54670 0.9811683893203735
73628 1.0861322283744812
15660 0.9862233400344849
92236 1.0091512203216553


In [39]:
top1dict_reordered

{2440: 34463}